# Reading the coronavirus from Sciensano with Python

**Author:** Pierre de Buyl  
**Date:** 14 september 2020  
**Licence:** [CC-BY](https://creativecommons.org/licenses/by/4.0/)

In this notebook, I present how to load the files published by Sciensano at
https://epistat.wiv-isp.be/covid/

The files are available in csv and present a non-unique index as the lines
correspond to a set of date/province (for hospitalization data) and
date/province/agegroup/sex for the cases.

I use [pandas](https://pandas.pydata.org/) for its read_csv, groupby and rolling mean features.
Also [matplotlib](https://matplotlib.org/) and [NumPy](https://numpy.org/).

See https://github.com/pdebuyl/coronavirus_notebooks for the notebook file. You can
execute the notebook online at https://mybinder.org/v2/gh/pdebuyl/coronavirus_notebooks/master

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os
import os.path

In [ ]:
# Give room for the tilted date labels
plt.rcParams['figure.subplot.bottom'] = 0.2
plt.rcParams['figure.subplot.hspace'] = 0.3

In [ ]:
# Change the line below to the location of the CSV files
data_directory = 'data'

# Read hospitalisation and cases data
# File encoding must be set, the default from pandas is utf-8

df_HOSP =  pd.read_csv(os.path.join(data_directory, 'COVID19BE_HOSP.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE'])

df_CASES = pd.read_csv(os.path.join(data_directory, 'COVID19BE_CASES_AGESEX.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE', 'AGEGROUP', 'SEX'])

df_TESTS = pd.read_csv(os.path.join(data_directory, 'COVID19BE_tests.csv'),
                       encoding='utf-8', index_col=['DATE', 'PROVINCE'])

df_MORT = pd.read_csv(os.path.join(data_directory, 'COVID19BE_MORT.csv'),
                       encoding='utf-8', index_col=['DATE', 'REGION', 'AGEGROUP', 'SEX'])


## Plotting the data

I consider the hospitalization data for all provinces summed, as it is
the main reporting done in the media and in the comparisons with epidemiologic
models.

Some data are daily values, such as the number of *new* hospitalizations. Others
are cumulative, i.e. the sum to date for every data point, such as cumulative cases.

In [ ]:
HOSP_bydate = df_HOSP.groupby('DATE').sum()

CASES_bydate = df_CASES.groupby('DATE').sum()

TESTS_bydate = df_TESTS.groupby('DATE').sum() #[:-1]

HOSP_time = np.asarray(HOSP_bydate.index, dtype=np.datetime64)
CASES_time = np.asarray(CASES_bydate.index, dtype=np.datetime64)
TESTS_time = np.asarray(TESTS_bydate.index, dtype=np.datetime64)


In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN'], label='in hospital')

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN_ICU'], label='in ICU')

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.legend()
plt.yscale('log')
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.title("COVID19 Belgium - hospital occupancy\nData source: Sciensano - Figure: P. de Buyl")

In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN']/HOSP_bydate['TOTAL_IN_ICU'])

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.title("COVID19 Belgium - ratio of total / ICU beds")

In [ ]:
plt.figure()

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN'], label='in hospital')

plt.plot(HOSP_time, HOSP_bydate['TOTAL_IN_ICU'], label='in ICU')

plt.plot(CASES_time, CASES_bydate['CASES'], label='cases')
plt.plot(CASES_time, np.cumsum(CASES_bydate['CASES']), label='cumul cases')

plt.plot(TESTS_time, TESTS_bydate['TESTS_ALL'], label='tests')

plt.axhline(2200)

xt = plt.xticks()
plt.xticks(xt[0][::1])
plt.legend()
plt.yscale('log')
plt.grid()
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=30, ha="right");

plt.title('Daily indicators and cumulative cases')

## Plotting the cases and hospitalization numbers per province

In [ ]:
cases_prov = df_CASES.groupby(level=[0,1]).sum().groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(cases_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = cases_prov.get_group(k)
    g = g.reset_index(level=1, drop=True)
    g.rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Cases per province, rolling mean')

In [ ]:
hosp_prov = df_HOSP.groupby('PROVINCE') #.reset_index(level=1, drop=True)

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(hosp_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = hosp_prov.get_group(k).reset_index(level=1, drop=True)
    g['TOTAL_IN'].rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    g['TOTAL_IN_ICU'].rolling(7).mean().plot(ax=ax, legend=i==1, rot=45, ls='--')
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Hospitalization per province, rolling mean')

In [ ]:
tests_prov = df_TESTS.groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(tests_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    g = tests_prov.get_group(k).reset_index(level=1, drop=True)
    g.rolling(7).mean().plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Tests per province, rolling mean')

In [ ]:
tests_prov = df_TESTS.groupby('PROVINCE')
cases_prov = df_CASES.groupby('PROVINCE')

fig, axes = plt.subplots(4, 3, figsize=(9, 9), sharex=True, sharey=True)

for i, k in enumerate(tests_prov.indices.keys()):
    i = i if i < 2 else i+1
    ix = i // 3
    iy = i % 3
    ax = axes[ix][iy]
    plt.sca(ax)
    tests_group = tests_prov.get_group(k).reset_index(level=1, drop=True).groupby('DATE').sum().rolling(7).mean()[1:-1]
    cases_group = cases_prov.get_group(k).groupby('DATE').sum().rolling(7).mean()
    positivity = cases_group['CASES']/tests_group['TESTS_ALL']
    positivity.plot(ax=ax, legend=i==1, rot=45)
    plt.yscale('log')
    plt.title(k)
    plt.grid()

plt.suptitle('Positivity per province, rolling mean')

In [ ]:
plt.figure()

TESTS_rm = TESTS_bydate['TESTS_ALL'].rolling(7).mean()
CASES_rm = CASES_bydate['CASES'].rolling(7).mean()

plt.plot(TESTS_time, CASES_rm/TESTS_rm)

plt.yscale('log')
plt.grid()

plt.title('Positive rate of tests')

In [ ]:
plt.figure()

TESTS_rm = TESTS_bydate['TESTS_ALL'].rolling(7).mean()

plt.plot(TESTS_time, TESTS_rm)

plt.yscale('log')
plt.grid()

plt.title('Number of tests, rolling mean')

In [ ]:
deaths_all = df_MORT.groupby(level=[0,1,2]).sum().reset_index(level=(1,2), drop=True)

deaths_all.rolling(7).mean().plot(rot=45)

plt.yscale('log')

plt.title("Daily deaths, rolling mean")

In [ ]:
deaths_agegroup = df_MORT.groupby(level=[0,2]).sum().groupby('AGEGROUP')

for i, k in enumerate(deaths_agegroup.indices.keys()):
    if i==0: continue # First range contains no recorded death
    print(k)
    g = deaths_agegroup.get_group(k)
    g = g.reset_index(level=1, drop=True)
    g.rolling(7).mean().plot(rot=45, label='rolling mean')
    ax = plt.gca()
    g.plot(ax=ax)
    plt.yscale('log')
    plt.title(f'Arge range {k}')
    plt.grid()
